In [332]:
import os

import zarr
import numpy as np
import xarray as xr
import torch
from tqdm import tqdm

from src.era5_dataset import ERA5Dataset, TimeMode
from src.fuxi_ligthning import FuXi
from torch.utils.data import DataLoader
%load_ext autoreload
%autoreload
# Batch Size darf nicht größer als 1 gewählt werden, sonst funktioniert die Logik unten beim schreiben nicht

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [333]:
bs = 1
autoregression_steps = 2
timesteps_cnt = 24
levels_cnt = 2
vars_cnt = 5
lats_cnt = 121
lons_cnt = 240
start_time = "2011-01-01T00:00:00"
end_time = "2011-01-07T18:00:00"
model_path = "/Users/xgxtphg/Documents/git/DL4WeatherAndClimate/checkpoints/epoch=0-step=2917.ckpt"



In [334]:
model = FuXi.load_from_checkpoint(model_path)
model: FuXi
model.set_autoregression_steps(autoregression_steps)
dataset = test_ds = ERA5Dataset(
    "/Users/xgxtphg/Documents/git/DL4WeatherAndClimate/data/era5_6hourly.zarr",
    TimeMode.BETWEEN,
    start_time="2011-01-01T00:00:00",
    end_time="2011-01-07T18:00:00",
    max_autoregression_steps=autoregression_steps,
    zarr_col_names='lessig'
)
dl = DataLoader(dataset, batch_size=bs, shuffle=False, num_workers=os.cpu_count() // 2, pin_memory=True)


In [335]:
store = zarr.DirectoryStore('./preds.zarr')
root = zarr.group(store=store, overwrite=True)

latitude = root.create_dataset('latitude', shape=(0,), chunks=(lats_cnt,), dtype=np.float64, fill_value=-99999)
levels = root.create_dataset('level', shape=(0,), chunks=(levels_cnt,), dtype=np.int32, fill_value=-99)
longitude = root.create_dataset('longitude', shape=(0,), chunks=(lons_cnt,), dtype=np.float64, fill_value=-99999)
pred_timedelta = root.create_dataset('prediction_timedelta', shape=(0,), chunks=(autoregression_steps,),
                                     dtype='timedelta64[ns]')
# die Zeit muss noch richtig gesetzt werden, wahrscheinlich dann über das Dataset
time = root.create_dataset('time', shape=(0,), chunks=(timesteps_cnt,), dtype='datetime64[ns]', fill_value=-99999)

temp = root.create_dataset('temperature', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt),
                           dtype=np.float64,
                           chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
temp.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'level', 'latitude', 'longitude']
humid = root.create_dataset('specific_humidity', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt),
                            dtype=np.float64,
                            chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
humid.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'level', 'latitude', 'longitude']
uwind = root.create_dataset('u_component_of_wind', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt),
                            dtype=np.float64,
                            chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
uwind.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'level', 'latitude', 'longitude']
vwind = root.create_dataset('v_component_of_wind', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt),
                            dtype=np.float64,
                            chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
vwind.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'level', 'latitude', 'longitude']
geo = root.create_dataset('geopotential', shape=(0, autoregression_steps, levels_cnt, lats_cnt, lons_cnt),
                          dtype=np.float64,
                          chunks=(16, autoregression_steps, levels_cnt, lats_cnt, lons_cnt), fill_value=-99999)
geo.attrs['_ARRAY_DIMENSIONS'] = ['time', 'prediction_timedelta', 'level', 'latitude', 'longitude']


In [336]:
latitude.append(np.linspace(-90, 90, lats_cnt))
latitude.attrs['_ARRAY_DIMENSIONS'] = ['latitude']

levels.append([500, 850])
levels.attrs['_ARRAY_DIMENSIONS'] = ['level']

longitude.append(np.linspace(-180, 180, lons_cnt))
longitude.attrs['_ARRAY_DIMENSIONS'] = ['longitude']

timedelta = [np.timedelta64(6 * i, 'h') for i in range(autoregression_steps)]
pred_timedelta.append(np.array(timedelta))
pred_timedelta.attrs['_ARRAY_DIMENSIONS'] = ['prediction_timedelta']

times = [np.datetime64(start_time) + i * timedelta[1] for i in range(timesteps_cnt)]
time.append(times)
time.attrs['_ARRAY_DIMENSIONS'] = ['time']

In [337]:
# range muss noch durch den dataloader ersetzt werden
for idx, batch in tqdm(enumerate(dl)):
    out = model.forward(batch)[0, :, :, :, :]
    preds = torch.reshape(out, (autoregression_steps, vars_cnt, out.shape[1] // vars_cnt, lats_cnt, lons_cnt)).numpy()
    temp.append(preds[None, :, 0, 2:4, :, :], axis=0)
    humid.append(preds[None, :, 1, 2:4, :, :], axis=0)
    uwind.append(preds[None, :, 2, 2:4, :, :], axis=0)
    vwind.append(preds[None, :, 3, 2:4, :, :], axis=0)
    geo.append(preds[None, :, 4, 2:4, :, :], axis=0)

24it [00:14,  1.65it/s]


In [338]:
zarr.consolidate_metadata(store)

<zarr.hierarchy.Group '/'>

In [339]:
# beim laden mit xarray kommt es zum problem, dass 0´en durch nan ersetzt werden
b = xr.open_zarr('./preds.zarr')
use_dask = True
by_init = False
obs = xr.open_zarr('./preds.zarr', chunks='auto')


In [340]:
obs

<xarray.Dataset>
Dimensions:               (time: 24, prediction_timedelta: 2, level: 2,
                           latitude: 121, longitude: 240)
Coordinates:
  * latitude              (latitude) float64 -90.0 -88.5 -87.0 ... 88.5 90.0
  * level                 (level) float64 500.0 850.0
  * longitude             (longitude) float64 -180.0 -178.5 ... 178.5 180.0
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 00:00:00 06:...
  * time                  (time) datetime64[ns] 2011-01-01 ... 2011-01-06T18:...
Data variables:
    geopotential          (time, prediction_timedelta, level, latitude, longitude) float64 dask.array<chunksize=(16, 2, 2, 121, 240), meta=np.ndarray>
    specific_humidity     (time, prediction_timedelta, level, latitude, longitude) float64 dask.array<chunksize=(16, 2, 2, 121, 240), meta=np.ndarray>
    temperature           (time, prediction_timedelta, level, latitude, longitude) float64 dask.array<chunksize=(16, 2, 2, 121, 240), meta=np.ndarray>
    u_component_of_wind   (time, prediction_timedelta, level, latitude, longitude) float64 dask.array<chunksize=(16, 2, 2, 121, 240), meta=np.ndarray>
    v_component_of_wind   (time, prediction_timedelta, level, latitude, longitude) float64 dask.array<chunksize=(16, 2, 2, 121, 240), meta=np.ndarray>

In [341]:
forecast_path = "/Users/xgxtphg/Documents/git/DL4WeatherAndClimate/notebooks/preds.zarr"
obs_path = "/Users/xgxtphg/Documents/git/DL4WeatherAndClimate/notebooks/preds.zarr"

In [342]:
from weatherbench2 import config

paths = config.Paths(
    forecast=forecast_path,
    obs=obs_path,
    output_dir='./',  # Directory to save evaluation results
)

In [343]:
selection = config.Selection(
    variables=[
        'geopotential',
        'temperature'
    ],
    levels=[500, 850],
    time_slice=slice('2011-01-01', '2011-01-05'),
)

In [344]:
data_config = config.Data(selection=selection, paths=paths)

In [345]:
from weatherbench2.regions import SliceRegion, ExtraTropicalRegion
from weatherbench2.metrics import MSE

regions = {
    'global': SliceRegion(),
    'tropics': SliceRegion(lat_slice=slice(-20, 20)),
    'extra-tropics': ExtraTropicalRegion(),
}

eval_configs = {
  'deterministic': config.Eval(
      metrics={
          'mse': MSE(), 
      },
      regions=regions
  )
}

In [346]:
print(1)

1


In [347]:
from weatherbench2.evaluation import evaluate_in_memory, evaluate_with_beam

In [348]:
evaluate_in_memory(data_config, eval_configs)   # Takes around 5 minutes

In [349]:
results = xr.open_dataset('./deterministic.nc')
results['level'] = results['level'].astype('int32')
results

<xarray.Dataset>
Dimensions:               (region: 3, level: 2, lead_time: 2,
                           prediction_timedelta: 2, metric: 1)
Coordinates:
  * region                (region) object 'global' 'tropics' 'extra-tropics'
  * level                 (level) int32 500 850
  * lead_time             (lead_time) timedelta64[ns] 00:00:00 06:00:00
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 00:00:00 06:...
  * metric                (metric) object 'mse'
Data variables:
    geopotential          (metric, region, lead_time, level, prediction_timedelta) float64 ...
    temperature           (metric, region, lead_time, level, prediction_timedelta) float64 ...

In [350]:
results = xr.concat(
    [
    results,
    results.sel(metric=['mse']).assign_coords(metric=['rmse']) ** 0.5
    ],
    dim='metric'
)

In [351]:
results['geopotential']

<xarray.DataArray 'geopotential' (metric: 2, region: 3, lead_time: 2, level: 2,
                                  prediction_timedelta: 2)>
array([[[[[0.00000000e+00, 7.92678143e-05],
          [0.00000000e+00, 1.33114204e-05]],

         [[7.18485662e-05, 1.02801048e-04],
          [1.29982228e-05, 1.12778369e-05]]],


        [[[0.00000000e+00, 6.95834822e-05],
          [0.00000000e+00, 1.40023079e-05]],

         [[6.13390285e-05, 7.64587017e-05],
          [1.36849310e-05, 9.33077558e-06]]],


        [[[0.00000000e+00, 8.43939803e-05],
          [0.00000000e+00, 1.29457158e-05]],

         [[7.74115348e-05, 1.16744730e-04],
          [1.26347305e-05, 1.23084666e-05]]]],

...

       [[[[0.00000000e+00, 8.90324740e-03],
          [0.00000000e+00, 3.64848193e-03]],

         [[8.47635335e-03, 1.01390852e-02],
          [3.60530482e-03, 3.35824909e-03]]],


        [[[0.00000000e+00, 8.34167143e-03],
          [0.00000000e+00, 3.74196578e-03]],

         [[7.83192368e-03, 8.74406666e-03],
          [3.69931494e-03, 3.05463182e-03]]],


        [[[0.00000000e+00, 9.18661964e-03],
          [0.00000000e+00, 3.59801554e-03]],

         [[8.79838251e-03, 1.08048475e-02],
          [3.55453661e-03, 3.50834243e-03]]]]])
Coordinates:
  * region                (region) object 'global' 'tropics' 'extra-tropics'
  * level                 (level) int32 500 850
  * lead_time             (lead_time) timedelta64[ns] 00:00:00 06:00:00
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 00:00:00 06:...
  * metric                (metric) object 'mse' 'rmse'

In [352]:
results['geopotential'].sel(metric='rmse', level=500, region='global').plot();

DTypePromotionError: The DType <class 'numpy.dtypes.TimeDelta64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.TimeDelta64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>)

Error in callback <function _draw_all_if_interactive at 0x30a8f6660> (for post_execute), with arguments args (),kwargs {}:


TypeError: Cannot cast array data from dtype('<m8[ns]') to dtype('float64') according to the rule 'safe'

TypeError: Cannot cast array data from dtype('<m8[ns]') to dtype('float64') according to the rule 'safe'

<Figure size 640x480 with 1 Axes>

In [353]:
results['geopotential']

<xarray.DataArray 'geopotential' (metric: 2, region: 3, lead_time: 2, level: 2,
                                  prediction_timedelta: 2)>
array([[[[[0.00000000e+00, 7.92678143e-05],
          [0.00000000e+00, 1.33114204e-05]],

         [[7.18485662e-05, 1.02801048e-04],
          [1.29982228e-05, 1.12778369e-05]]],


        [[[0.00000000e+00, 6.95834822e-05],
          [0.00000000e+00, 1.40023079e-05]],

         [[6.13390285e-05, 7.64587017e-05],
          [1.36849310e-05, 9.33077558e-06]]],


        [[[0.00000000e+00, 8.43939803e-05],
          [0.00000000e+00, 1.29457158e-05]],

         [[7.74115348e-05, 1.16744730e-04],
          [1.26347305e-05, 1.23084666e-05]]]],

...

       [[[[0.00000000e+00, 8.90324740e-03],
          [0.00000000e+00, 3.64848193e-03]],

         [[8.47635335e-03, 1.01390852e-02],
          [3.60530482e-03, 3.35824909e-03]]],


        [[[0.00000000e+00, 8.34167143e-03],
          [0.00000000e+00, 3.74196578e-03]],

         [[7.83192368e-03, 8.74406666e-03],
          [3.69931494e-03, 3.05463182e-03]]],


        [[[0.00000000e+00, 9.18661964e-03],
          [0.00000000e+00, 3.59801554e-03]],

         [[8.79838251e-03, 1.08048475e-02],
          [3.55453661e-03, 3.50834243e-03]]]]])
Coordinates:
  * region                (region) object 'global' 'tropics' 'extra-tropics'
  * level                 (level) int32 500 850
  * lead_time             (lead_time) timedelta64[ns] 00:00:00 06:00:00
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 00:00:00 06:...
  * metric                (metric) object 'mse' 'rmse'